<a href="https://colab.research.google.com/github/nhamhung/CS3244-Machine-Learning/blob/main/CS3244_DL_LSTM_Pretrained_Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import json 
import os
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk

X_train = pd.read_csv('/content/drive/My Drive/CS3244 Project/Wiki Toxic Classification/train.csv')
X_test = pd.read_csv('/content/drive/My Drive/CS3244 Project/Wiki Toxic Classification/test.csv')
df_test_labels = pd.read_csv('/content/drive/My Drive/CS3244 Project/Wiki Toxic Classification/test_labels.csv')

# Pre-trained Embeddings

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D,Bidirectional
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
import matplotlib.pyplot as plt
%matplotlib inline
import gensim.models.keyedvectors as word2vec
import gc

In [ ]:
# preprocess
def preprocess(max_features=20000, maxlen=200):
  labels = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
  
  y = X_train[labels].values
  train_sentence_list = X_train['comment_text']
  test_sentence_list = X_test['comment_text']

  tokenizer = Tokenizer(num_words=max_features)
  tokenizer.fit_on_texts(list(train_sentence_list))
  train_tokenized_list = tokenizer.texts_to_sequences(train_sentence_list)
  test_tokenized_list = tokenizer.texts_to_sequences(test_sentence_list)

  X_t = pad_sequences(train_tokenized_list, maxlen=maxlen)
  X_te = pad_sequences(test_tokenized_list, maxlen=maxlen)

  return tokenizer, X_t, X_te, train_tokenized_list, test_tokenized_list, y

In [ ]:
tokenizer, X_t, X_te, train_tokenized_list, test_tokenized_list, y = preprocess()

In [ ]:
def loadEmbeddingMatrix(typeToLoad="word2vec"):
        word2vecDict = word2vec.KeyedVectors.load_word2vec_format("/content/drive/My Drive/CS3244 Project/GoogleNews-vectors-negative300.bin", binary=True)
        embed_size = 300

        embeddings_index = dict()
        for word in word2vecDict.wv.vocab:
            embeddings_index[word] = word2vecDict.word_vec(word)
            
        gc.collect()

        #We get the mean and standard deviation of the embedding weights so that we could maintain the 
        #same statistics for the rest of our own random generated weights. 
        all_embs = np.stack(list(embeddings_index.values()))
        emb_mean, emb_std = all_embs.mean(), all_embs.std()
        
        nb_words = len(tokenizer.word_index)
        
        #We are going to set the embedding size to the pretrained dimension as we are replicating it.
        #the size will be Number of Words in Vocab X Embedding Size
        embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
        gc.collect()

        #With the newly created embedding matrix, we'll fill it up with the words that we have in both 
        #our own dictionary and loaded pretrained embedding. 
        embeddedCount = 0
        for word, i in tokenizer.word_index.items():
            i-=1
            #then we see if this word is in word2vec's dictionary, if yes, get the corresponding weights
            embedding_vector = embeddings_index.get(word)
            #and store inside the embedding matrix that we will train later on.
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
                embeddedCount+=1

        print('total embedded:',embeddedCount,'common words')
        
        del(embeddings_index)
        gc.collect()
        
        #finally, return the embedding matrix
        return embedding_matrix

In [ ]:
embedding_matrix = loadEmbeddingMatrix()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


total embedded: 66078 common words


In [ ]:
# list(embedding_matrix.keys())[:100]

In [ ]:
embedding_matrix.shape

(210337, 300)

In [ ]:
def build_pretrained_embeddings_LSTM(maxlen):
  inp = Input(shape=(maxlen, )) #maxlen=200 as defined earlier  
  x = Embedding(len(tokenizer.word_index), embedding_matrix.shape[1],weights=[embedding_matrix],trainable=False)(inp)
  x = Bidirectional(LSTM(60, return_sequences=True,name='lstm_layer',dropout=0.2,recurrent_dropout=0.2))(x)  
  x = GlobalMaxPool1D()(x)
  x = Dropout(0.2)(x)
  x = Dense(50, activation="relu")(x)
  x = Dropout(0.2)(x)
  x = Dense(6, activation="sigmoid")(x)
  model = Model(inputs=inp, outputs=x)
  model.compile(loss='binary_crossentropy',
                    optimizer='adam',
                  metrics=['accuracy', tf.keras.metrics.AUC()])
  return model

In [ ]:
model = build_pretrained_embeddings_LSTM(200)
model.summary()


Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 300)          63101100  
_________________________________________________________________
bidirectional (Bidirectional (None, 200, 120)          173280    
_________________________________________________________________
global_max_pooling1d (Global (None, 120)               0         
_________________________________________________________________
dropout (Dropout)            (None, 120)               0         
_________________________________________________________________
dense (Dense)                (None, 50)                6050      
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)               

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
batch_size = 32
epochs = 4


checkpoint_filepath = '/content/drive/My Drive/CS3244 Project/checkpoints/checkpoint_pretrained'

callbacks_list = [EarlyStopping(monitor='val_loss', patience=3), ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_loss', mode='min', save_best_only=True)]

hist = model.fit(X_t,y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=callbacks_list)

Epoch 1/4
4488/4488 [==============================] - 2193s 489ms/step - loss: 0.0847 - accuracy: 0.9513 - auc: 0.9302 - val_loss: 0.0620 - val_accuracy: 0.9940 - val_auc: 0.9668
Epoch 2/4
4488/4488 [==============================] - 2328s 519ms/step - loss: 0.0602 - accuracy: 0.9941 - auc: 0.9671 - val_loss: 0.0554 - val_accuracy: 0.9940 - val_auc: 0.9746
Epoch 3/4
4488/4488 [==============================] - 2363s 527ms/step - loss: 0.0546 - accuracy: 0.9941 - auc: 0.9739 - val_loss: 0.0527 - val_accuracy: 0.9940 - val_auc: 0.9764
Epoch 4/4
4488/4488 [==============================] - 2413s 538ms/step - loss: 0.0514 - accuracy: 0.9938 - auc: 0.9771 - val_loss: 0.0513 - val_accuracy: 0.9940 - val_auc: 0.9753


In [ ]:
model.save('word2vec_pretrained.h5')

In [ ]:
hist.history

{'accuracy': [0.9512648582458496,
  0.9940882921218872,
  0.9940743446350098,
  0.9938237071037292],
 'auc': [0.9302154183387756,
  0.9670933485031128,
  0.9738765954971313,
  0.9770700335502625],
 'loss': [0.0846811905503273,
  0.060181282460689545,
  0.05460897833108902,
  0.05144799128174782],
 'val_accuracy': [0.9939842224121094,
  0.9939842224121094,
  0.9939842224121094,
  0.9939842224121094],
 'val_auc': [0.9668133854866028,
  0.9746487140655518,
  0.9763771295547485,
  0.9753377437591553],
 'val_loss': [0.06196773424744606,
  0.05535469576716423,
  0.05267029255628586,
  0.05130878835916519]}

In [ ]:
y_test = model.predict([X_te], batch_size=1024, verbose=1)


150/150 [==============================] - 95s 635ms/step


In [ ]:
y_test

array([[9.9871749e-01, 3.8115072e-01, 9.3926477e-01, 1.7975929e-01,
        8.5079920e-01, 4.1678932e-01],
       [2.6464164e-03, 7.3839619e-06, 3.0630827e-04, 1.7589326e-05,
        2.5910139e-04, 1.1770344e-04],
       [4.1717887e-03, 3.2480046e-05, 7.5638294e-04, 4.4749981e-05,
        4.0894747e-04, 2.8198957e-04],
       ...,
       [3.4181178e-03, 1.6401818e-06, 2.9161572e-04, 3.5324622e-06,
        2.2608042e-04, 9.2848706e-05],
       [2.2335052e-03, 1.3719061e-06, 1.1910328e-04, 1.3414359e-05,
        1.3443828e-04, 1.2895465e-04],
       [9.3819833e-01, 6.8089426e-02, 7.1672374e-01, 4.5990348e-02,
        5.3433204e-01, 8.8872850e-02]], dtype=float32)

In [ ]:
y_test
sample_submission = pd.read_csv('/content/drive/My Drive/CS3244 Project/Wiki Toxic Classification/sample_submission.csv')
sample_submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_test
sample_submission.to_csv('pretrained_LSTM_super_final.csv', index=False)
# sample_submission = pd.merge(sample_submission, df_test, how='inner', on='id')
sample_submission

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.998717,3.811507e-01,0.939265,0.179759,0.850799,0.416789
1,0000247867823ef7,0.002646,7.383962e-06,0.000306,0.000018,0.000259,0.000118
2,00013b17ad220c46,0.004172,3.248005e-05,0.000756,0.000045,0.000409,0.000282
3,00017563c3f7919a,0.001019,9.753929e-07,0.000077,0.000003,0.000079,0.000037
4,00017695ad8997eb,0.006753,3.096107e-05,0.000882,0.000062,0.000796,0.000403
...,...,...,...,...,...,...,...
153159,fffcd0960ee309b5,0.614291,2.680689e-03,0.273463,0.000836,0.117809,0.010365
153160,fffd7a9a6eb32c16,0.016292,2.881332e-05,0.001742,0.000052,0.001712,0.000707
153161,fffda9e8d6fafa9e,0.003418,1.640182e-06,0.000292,0.000004,0.000226,0.000093
153162,fffe8f1340a79fc2,0.002234,1.371906e-06,0.000119,0.000013,0.000134,0.000129
